<a href="https://colab.research.google.com/github/sonavmorris/MAIN-PROJECT-AUTHENTICATION-OF-AN-INDIVIDUAL-USING-IRIS-RECOGNITION/blob/main/iris%20recognition%20system%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
main_folder="/content/drive/MyDrive/Main project"
class_names=os.listdir(main_folder)
print(class_names)


['Ali Habib', 'Mohamed Bebo', 'Mohamed Labib', 'Ahmed Amr', 'Mohamed Mokhtar']


In [4]:
validation_folder="/content/drive/MyDrive/Main project"
val_class_names=os.listdir(validation_folder)
print(val_class_names)

['Ali Habib', 'Mohamed Bebo', 'Mohamed Labib', 'Ahmed Amr', 'Mohamed Mokhtar']


In [5]:
from PIL import Image

In [6]:
data_folder = "/content/drive/MyDrive/Main project"

def process_files(folder):
    for entry in os.scandir(folder):
        if entry.is_file() and entry.name.endswith('.jpg'):
            image = Image.open(entry.path)
            image = image.resize((256, 256))
            image = image.convert('L')
            image.save(entry.path)
        elif entry.is_dir():
            process_files(entry.path)

process_files(data_folder) 

In [8]:
print(data_folder)

/content/drive/MyDrive/Main project


In [18]:
import cv2
import numpy as np

In [19]:
def segment_iris(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)
    circles = cv2.HoughCircles(edges, cv2.HOUGH_GRADIENT, 1, 20,
                               param1=50, param2=30, minRadius=0, maxRadius=0)
    iris = None

    
    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")
        for (x, y, r) in circles:
            cv2.circle(image, (x, y), r, (0, 255, 0), 2)
            iris = image[y-r:y+r, x-r:x+r]

    return iris
    

In [20]:
for root, dirs, files in os.walk(data_folder):
    for filename in files:
        if filename.endswith('.jpg'):
          
            image = cv2.imread(os.path.join(root, filename))

            iris = segment_iris(image)
            if iris is not None:
                cv2.imwrite(os.path.join(root, 'iris_'+filename), iris)


In [21]:
from skimage.feature import local_binary_pattern

def extract_lbp_features(image):
    radius = 1
    n_points = 8 * radius
    lbp = local_binary_pattern(image, n_points, radius, method='uniform')
    return lbp.flatten()